<a href="https://colab.research.google.com/github/samanthajmichael/ml_project/blob/main/notebooks/Base_Model_Training_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install tensorflow ffmpeg-python opencv-python matplotlib

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [4]:
# Check if GPU is available
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# Get GPU device information
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    for device in gpu_devices:
        # Get device details
        device_details = tf.config.experimental.get_device_details(device)
        print("GPU Details:", device_details)

GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU Details: {'compute_capability': (7, 5), 'device_name': 'Tesla T4'}


In [5]:
def load_and_preprocess_frame(frame_data, target_size=(224, 224)):
    # Convert to TensorFlow tensors earlier in the pipeline
    nparr = np.frombuffer(frame_data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img.astype(np.float32) / 255.0
    # Convert to TensorFlow tensor here
    return tf.convert_to_tensor(img)

def data_generator(df, batch_size):
    while True:
        batch = df.sample(n=batch_size)
        # Create tensor batch directly
        frames = tf.stack([load_and_preprocess_frame(frame_data)
                         for frame_data in batch['frame_data']])
        yield frames

@tf.function  # Add tf.function for faster execution
def augment_frame(frame):
    # Ensure input is a tensor
    frame = tf.convert_to_tensor(frame)
    frame = tf.image.random_flip_left_right(frame)
    frame = tf.image.random_brightness(frame, max_delta=0.1)
    frame = tf.image.random_contrast(frame, lower=0.9, upper=1.1)
    frame = tf.clip_by_value(frame, 0.0, 1.0)
    return frame

def create_base_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    # Add batch normalization for better training stability
    x = tf.keras.layers.BatchNormalization()(x)
    projection_head = Dense(128, activation='relu')(x)
    # Add L2 normalization layer with explicit output shape
    projection_head = tf.keras.layers.Lambda(
        lambda x: tf.math.l2_normalize(x, axis=1),
        output_shape=lambda input_shape: input_shape  # Explicitly specify output shape
    )(projection_head)
    return Model(inputs=base_model.input, outputs=projection_head)

@tf.function
def nt_xent_loss(z_i, z_j, temperature=0.5):
    # Ensure inputs are float32
    z_i = tf.cast(z_i, tf.float32)
    z_j = tf.cast(z_j, tf.float32)

    z_i = tf.math.l2_normalize(z_i, axis=1)
    z_j = tf.math.l2_normalize(z_j, axis=1)
    similarity_matrix = tf.matmul(z_i, z_j, transpose_b=True) / temperature

    batch_size = tf.shape(z_i)[0]
    contrastive_labels = tf.range(batch_size)

    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=contrastive_labels,
        logits=similarity_matrix
    )
    return tf.cast(tf.reduce_mean(loss), tf.float32)

@tf.function
def train_step(model, optimizer, x):
    x_i = augment_frame(x)
    x_j = augment_frame(x)

    with tf.GradientTape() as tape:
        z_i = model(x_i, training=True)
        z_j = model(x_j, training=True)
        loss = nt_xent_loss(z_i, z_j)
        # Ensure loss is float32
        loss = tf.cast(loss, tf.float32)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

def train_contrastive_model(df, epochs=1, batch_size=15, steps_per_epoch=50):
    # Use float32 instead of mixed precision to avoid dtype issues
    model = create_base_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    generator = tf.data.Dataset.from_generator(
        lambda: data_generator(df, batch_size),
        output_signature=(
            tf.TensorSpec(shape=(batch_size, 224, 224, 3), dtype=tf.float32)
        )
    ).prefetch(tf.data.AUTOTUNE)

    for epoch in range(epochs):
        # Initialize total_loss as float32
        total_loss = tf.constant(0.0, dtype=tf.float32)

        for step, x in enumerate(generator.take(steps_per_epoch)):
            loss = train_step(model, optimizer, x)
            # Ensure both are same dtype before adding
            total_loss += tf.cast(loss, tf.float32)

            if step % 10 == 0:
                tf.print(f"Epoch {epoch+1}/{epochs}, Step {step}, Loss: {loss:.4f}")

        avg_loss = total_loss / tf.cast(steps_per_epoch, tf.float32)
        tf.print(f"Epoch {epoch+1}/{epochs}, Avg Loss: {avg_loss:.4f}")

        if (epoch + 1) % 10 == 0:
            model.save(f'model_checkpoint_epoch_{epoch+1}', save_format='tf')

    return model

In [6]:
# Usage
df = pd.read_pickle("/content/drive/MyDrive/ML Project/middle_15min_frames.pkl")

In [7]:
print(df.info())
print(f"DataFrame shape: {df.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20700 entries, 0 to 20699
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   frame_number  20700 non-null  int64 
 1   frame_data    20700 non-null  object
dtypes: int64(1), object(1)
memory usage: 323.6+ KB
None
DataFrame shape: (20700, 2)


In [8]:
# Investigate frame numbers
print("\nFrame number statistics:")
print(df['frame_number'].describe())

# Check for duplicate frame numbers
duplicates = df['frame_number'].duplicated().sum()
print(f"\nNumber of duplicate frame numbers: {duplicates}")

# Display a few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(df.head())


Frame number statistics:
count     20700.000000
mean      92814.500000
std        5975.719622
min       82465.000000
25%       87639.750000
50%       92814.500000
75%       97989.250000
max      103164.000000
Name: frame_number, dtype: float64

Number of duplicate frame numbers: 0

First few rows of the DataFrame:
   frame_number                                         frame_data
0         82465  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1         82466  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2         82467  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3         82468  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4         82469  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


In [9]:
trained_model = train_contrastive_model(df, epochs=4, batch_size=30, steps_per_epoch=50)
trained_model.save('/content/drive/MyDrive/ML Project/trained_base_model_4_epochs.h5')
print("Training completed and model saved.")

Epoch 1/4, Step 0, Loss: 2.9318
Epoch 1/4, Step 10, Loss: 2.2292
Epoch 1/4, Step 20, Loss: 2.2271
Epoch 1/4, Step 30, Loss: 2.1055
Epoch 1/4, Step 40, Loss: 2.0562
Epoch 1/4, Avg Loss: 2.2067
Epoch 2/4, Step 0, Loss: 2.0387
Epoch 2/4, Step 10, Loss: 2.2244
Epoch 2/4, Step 20, Loss: 2.1796
Epoch 2/4, Step 30, Loss: 2.2425
Epoch 2/4, Step 40, Loss: 2.0691
Epoch 2/4, Avg Loss: 2.1328
Epoch 3/4, Step 0, Loss: 2.0479
Epoch 3/4, Step 10, Loss: 2.2810
Epoch 3/4, Step 20, Loss: 2.2711
Epoch 3/4, Step 30, Loss: 2.1422
Epoch 3/4, Step 40, Loss: 2.1256
Epoch 3/4, Avg Loss: 2.1572
Epoch 4/4, Step 0, Loss: 2.0949
Epoch 4/4, Step 10, Loss: 2.2712
Epoch 4/4, Step 20, Loss: 2.3063
Epoch 4/4, Step 30, Loss: 2.2005
Epoch 4/4, Step 40, Loss: 2.2037
Epoch 4/4, Avg Loss: 2.1899


Training completed and model saved.


In [10]:
trained_model

<Functional name=functional, built=True>